In [1]:
# =============================================================================
# Libraries
# =============================================================================

import pandas as pd
import numpy as np
import urllib.request
import collections
import os
import time
pd.options.mode.chained_assignment = None

# =============================================================================
# Data directories
# =============================================================================

dir_gaia = "db/src/Astro/"
dir_hip = "db/src/Astro/"
dir_simbad = "db/src/Astro/"
dir_rv = "db/src/RV/"
dir_results = "db/src/"

# =============================================================================
# Load Gaia database
# =============================================================================

t0 = time.time()

cols_gaia = ["source_id", "hip", "tycho2_id", "ra", "ra_error", "dec", "dec_error", "parallax", "parallax_error", 
             "pmra", "pmra_error", "pmdec", "pmdec_error", "ra_dec_corr", "ra_parallax_corr", "ra_pmra_corr", 
             "ra_pmdec_corr", "dec_parallax_corr", "dec_pmra_corr", "dec_pmdec_corr", "parallax_pmra_corr", 
             "parallax_pmdec_corr", "pmra_pmdec_corr", "phot_g_mean_flux", "phot_g_mean_flux_error", "phot_g_mean_mag", 
             "l", "b", "ecl_lon", "ecl_lat"]

gaia = pd.DataFrame()

for i in range(16):
    filename = dir_gaia + "TgasSource_000-000-0" + str(i).zfill(2) + ".csv.gz"
    gaia = gaia.append(pd.read_csv(filename, usecols=cols_gaia))

gaia.source_id = gaia.source_id.astype(str)
gaia.hip = gaia.hip.astype(str)
gaia.tycho2_id = gaia.tycho2_id.astype(str)

gaia["hip"] = gaia["hip"].map(lambda x: x.replace(".0", ""))
gaia.hip.replace("nan", "", inplace=True)
gaia.tycho2_id.replace("nan", "", regex=True, inplace=True)

n1 = len(gaia)
gaia = gaia[gaia.parallax > 0]
n2 = len(gaia)
print("\nDatabase Gaia-TGAS:", "\nOriginal objects:", n1,"\nDiscarded objects:", n1-n2,"\nFinal objects:", n2)

gaia_hip = gaia[gaia.hip != ""]
gaia_tyc = gaia[gaia.tycho2_id != ""]

# =============================================================================
# Load Hipparcos database
# =============================================================================

# Available in: http://cdsarc.u-strasbg.fr/viz-bin/Cat?cat=I%2F239&target=readme&#sRM2.1

names_hip={}
names_hip[1] = "hip"
names_hip[8] = "ra_hip"
names_hip[9] = "dec_hip"
names_hip[11] = "parallax_hip"
names_hip[12] = "pmra_hip"
names_hip[13] = "pmdec_hip"
names_hip[14] = "ra_error_hip"
names_hip[15] = "dec_error_hip"
names_hip[16] = "parallax_error_hip"
names_hip[17] = "pmra_error_hip"
names_hip[18] = "pmdec_error_hip"
names_hip[19] = "ra_dec_corr_hip"
names_hip[20] = "ra_parallax_corr_hip"
names_hip[21] = "dec_parallax_corr_hip"
names_hip[22] = "ra_pmra_corr_hip"
names_hip[23] = "dec_pmra_corr_hip"
names_hip[24] = "parallax_pmra_corr_hip"
names_hip[25] = "ra_pmdec_corr_hip"
names_hip[26] = "dec_pmdec_corr_hip"
names_hip[27] = "parallax_pmdec_corr_hip"
names_hip[28] = "pmra_pmdec_corr_hip"
names_hip[5] = "Vmag_hip"
names_hip[71] = "HenryDraperId_hip"
#names2_hip[76] = "sptype_hip"

names_hip = collections.OrderedDict(sorted(names_hip.items()))

hipparcos = pd.read_csv(dir_hip + "hip_main.dat.gz", delimiter="|", usecols=names_hip.keys(), names=names_hip.values())

# Columns format
cols = hipparcos.columns.tolist()
objects = ["hip", "HenryDraperId_hip"]
cols = [x for x in cols if x not in objects]
hipparcos[cols] = hipparcos[cols].apply(pd.to_numeric, errors='coerce')

hipparcos.hip = hipparcos.hip.astype(str)
hipparcos.hip = hipparcos.hip.map(lambda x: x.replace(".0", ""))

# Delete null astrometry values
n1 = len(hipparcos)
hipparcos.dropna(subset=["ra_hip", "dec_hip", "parallax_hip"], how="any", inplace=True)
n2 = len(hipparcos)
print("\nDatabase Hipparcos:", "\nOriginal objects:", n1,"\nDiscarded objects:", n1-n2,"\nFinal objects:", n2)

# =============================================================================
# Load Tycho database
# =============================================================================

# Available in: http://cdsarc.u-strasbg.fr/viz-bin/Cat?cat=I%2F239&target=readme&#sRM2.13

names_tyc={}
names_tyc[1] = "tycho2_id"
names_tyc[8] = "ra_tyc"
names_tyc[9] = "dec_tyc"
names_tyc[11] = "parallax_tyc"
names_tyc[12] = "pmra_tyc"
names_tyc[13] = "pmdec_tyc"
names_tyc[14] = "ra_error_tyc"
names_tyc[15] = "dec_error_tyc"
names_tyc[16] = "parallax_error_tyc"
names_tyc[17] = "pmra_error_tyc"
names_tyc[18] = "pmdec_error_tyc"
names_tyc[19] = "ra_dec_corr_tyc"
names_tyc[20] = "ra_parallax_corr_tyc"
names_tyc[21] = "dec_parallax_corr_tyc"
names_tyc[22] = "ra_pmra_corr_tyc"
names_tyc[23] = "dec_pmra_corr_tyc"
names_tyc[24] = "parallax_pmra_corr_tyc"
names_tyc[25] = "ra_pmdec_corr_tyc"
names_tyc[26] = "dec_pmdec_corr_tyc"
names_tyc[27] = "parallax_pmdec_corr_tyc"
names_tyc[28] = "pmra_pmdec_corr_tyc"
names_tyc[53] = "HenryDraperId_tyc"
names_tyc[5] = "Vmag_tyc"

names_tyc = collections.OrderedDict(sorted(names_tyc.items()))

tycho = pd.read_csv(dir_hip + "tyc_main.zip", delimiter="|", usecols = names_tyc.keys(), names = names_tyc.values())

# Split original tycho-id header which separated by white-space
tycho["a"], tycho["b"], tycho["c"] = tycho["tycho2_id"].str.split().str

# Concatenate tycho-id headers using "-"
tycho["tycho2_id"] = tycho["a"] + "-" + tycho["b"] + "-" + tycho["c"]

# Delete auxiliar columns used in conversion
del tycho["a"], tycho["b"], tycho["c"]

# Columns format
cols = tycho.columns.tolist()
objects = ["tycho2_id", "HenryDraperId_tyc"]
cols = [x for x in cols if x not in objects]
tycho[cols] = tycho[cols].apply(pd.to_numeric, errors='coerce')

# Delete null astrometry values
n1 = len(tycho)
tycho.dropna(subset=["ra_tyc", "dec_tyc", "parallax_tyc"], how="any", inplace=True)
n2 = len(tycho)
print("\nDatabase Tycho:", "\nOriginal objects:", n1,"\nDiscarded objects:", n1-n2,"\nFinal objects:", n2)

# =============================================================================
# Load Simbad-Hipparcos database
# =============================================================================

cols = ["typedident","identifier", "radvel", "coord1(ICRS,J2000/2000)", "plx", "pm", "MagV", "spec.type"]
simbad = pd.read_csv(dir_simbad + "simbad.zip", usecols=cols, delimiter="|")

# Modify ID
simbad["hip"] = simbad["typedident"].map(lambda x: str(x)[4:]).astype(str)
simbad["hip"] = simbad["hip"].str.strip()
del simbad["typedident"]

# Right ascension and declination format
# 1. Delete white-spaces on both sides of the text
simbad["coord1(ICRS,J2000/2000)"] = simbad["coord1(ICRS,J2000/2000)"].str.strip()

# 2. Split string into 6 values (hh:mm:ss for RA and hh:mm:ss for DEC)
simbad["ra_h"], simbad["ra_m"], simbad["ra_s"], simbad["dec_h"], simbad["dec_m"], simbad["dec_s"] = \
    simbad["coord1(ICRS,J2000/2000)"].str.split(" ").str

# 3. Concatenate the first 3 fields (RA) using conversion formule from hh:mm:ss to degrees
simbad["ra_simbad"] = simbad["ra_h"].astype(float)*15 + simbad["ra_m"].astype(float)/60 + simbad["ra_s"].astype(float)/3600

# 4. Concatenate the last 3 fields (DEC) using conversion formule from hh:mm:ss to degrees
simbad["dec_simbad"] = np.sign(simbad["dec_h"].astype(float)) * ( \
    np.abs(simbad["dec_h"].astype(float)) + simbad["dec_m"].astype(float)/60 + simbad["dec_s"].astype(float)/3600 )

# 5. Delete auxiliar columns used in conversion
del simbad["coord1(ICRS,J2000/2000)"]
del simbad["ra_h"], simbad["ra_m"], simbad["ra_s"], simbad["dec_h"], simbad["dec_m"], simbad["dec_s"]

# Proper motion format
# 1. Delete white-spaces on both sides of the text
simbad["pm"] = simbad["pm"].str.strip()

# 2. Split string into 2 values (pm_ra and pm_dec)
simbad["pmra_simbad"], simbad["pmdec_simbad"] = simbad["pm"].str.split(" ").str

# 3. Delete auxiliar columns used in conversion
del simbad["pm"]

# Modify name columns
simbad = simbad.rename(columns={"identifier": "name_simbad"})
simbad = simbad.rename(columns={"plx": "parallax_simbad"})   
simbad = simbad.rename(columns={"spec.type": "sptype_simbad"})     
simbad = simbad.rename(columns={"radvel": "RV_simbad"})    
simbad = simbad.rename(columns={"MagV": "Vmag_simbad"})

# Columns format
cols = simbad.columns.tolist()
objects = ["hip", "name_simbad", "sptype_simbad"]
cols = [x for x in cols if x not in objects]
simbad[cols] = simbad[cols].apply(pd.to_numeric, errors='coerce')

# Delete null astrometry values
n1 = len(simbad)
simbad.dropna(subset=["ra_simbad", "dec_simbad", "parallax_simbad"], how="any", inplace=True)      
n2 = len(simbad)

print("\nDatabase Simbad:", "\nOriginal objects:", n1,"\nDiscarded objects:", n1-n2,"\nFinal objects:", n2)

# =============================================================================
# Create database with unique astrometry values
# =============================================================================

print("\nBuilding database with just one astrometry source per star:")

# db = Gaia + Hipparcos
db = pd.DataFrame()
gaia["source_astro"] = "gaia"
exclusive_hip = hipparcos[~hipparcos.hip.isin(gaia_hip.hip)]
exclusive_hip.rename(columns=lambda x: x.replace("_hip", ""), inplace=True)
exclusive_hip["source_astro"] = "hipparcos"
db = pd.concat([gaia, exclusive_hip], axis=0)
print("Input from Hipparcos:", len(exclusive_hip))

# db = db + Tycho
exclusive_tyc = tycho[~tycho.tycho2_id.isin(gaia_tyc.tycho2_id)]
exclusive_tyc.rename(columns=lambda x: x.replace("_tyc", ""), inplace=True)
exclusive_tyc["source_astro"] = "tycho"
db = pd.concat([db, exclusive_tyc], axis=0)
print("Input from Tycho:", len(exclusive_tyc))

# db = db + Simbad
exclusive_simbad = simbad[~simbad.hip.isin(db.hip)]
exclusive_simbad.rename(columns=lambda x: x.replace("_simbad", ""), inplace=True)
exclusive_simbad = exclusive_simbad.loc[:,["hip", "ra", "dec", "parallax", "pmra", "pmdec"]]
exclusive_simbad["source_astro"] = "simbad"
db = pd.concat([db, exclusive_simbad], axis=0)
print("Input from Simbad:", len(exclusive_simbad))

# Clean np.NaN in objects produced by concat method
db.replace({"hip":{np.NaN:""},"tycho2_id":{np.NaN:""},"source_id":{np.NaN:""},"HenryDraperId":{np.NaN:""}}, inplace=True)

# Merge db + Simbad new columns

db = db.merge(simbad[["hip", "name_simbad","sptype_simbad","RV_simbad","Vmag_simbad"]], on="hip", how="left")

#db["id"] = db.hip.combine(db.tycho2_id, lambda x, y: x if x is not "" else y)
db["id"] = db.hip.astype(str) + db.tycho2_id.astype(str)

# Change the order of database columns
cols = ["id", "source_astro"] + cols_gaia + ["Vmag","HenryDraperId"] + \
       ["name_simbad","sptype_simbad","RV_simbad","Vmag_simbad"]
db = db[cols]
db = db.reset_index(drop=True)

print("Total lenght:", len(db))

# =============================================================================
# Full database with the information of all the catalogs
# =============================================================================

print("\nMerging astrometry databases (it keeps data of all sources):")

database = pd.DataFrame()

n1 = len(gaia)
print("Gaia objects (initial lenght of database):", n1)

database = gaia.merge(hipparcos, on="hip", how="outer")
n2 = len(database)
print("Hipparcos objects:", len(hipparcos), "| Input from Hipparcos:", n2-n1, "| Final lenght of database:", n2)

database = database.merge(tycho, on="tycho2_id", how="outer")
n3 = len(database)
print("Tycho objects:", len(tycho), "| Input from Tycho:", n3-n2, "| Final lenght of database:", n3)

database = database.merge(simbad, on="hip", how="outer")
n4 = len(database)
print("Simbad objects:", len(simbad),  "| Input from Simbad:", n4-n3, "| Final lenght of database:", n4)

#database["id"] = database.hip.combine(database.tycho2_id, lambda x, y: x if x is not "" else y)
database["id"] = database.hip.astype(str) + database.tycho2_id.astype(str)

database = database.reset_index(drop=True)

# =============================================================================
# Load radial velocities databases
# =============================================================================

t1 = time.time()
print("\nRadial velocity databases:")
radialv = {}

# =============================================================================
# RAVE-DR5
# =============================================================================

file = "RAVE-DR5.tsv"
cols = ["HRV", "e_HRV", "TGAS", "TYCHO2"]
delimiter = ";"

data = pd.DataFrame()
data = pd.DataFrame(pd.read_csv(dir_rv + file, usecols=cols, delimiter=delimiter, 
                                skiprows = list(np.arange(78)) + [79, 80]))

data.rename(columns={"HRV": "RV", "e_HRV": "e_RV", "TGAS": "source_id", "TYCHO2": "tycho2_id"}, inplace=True)
data = data.merge(gaia.loc[:,["source_id","hip"]], left_on="source_id", right_on="source_id", how="left")
data.source_id = data.source_id.str.strip()
data.tycho2_id = data.tycho2_id.str.strip()
data.hip.replace(np.nan,"", inplace=True)

data["source_rv"] = file
data["source_erv"] = "Catalogue"
data.source_erv[data.e_RV.isnull()] = "median"
median = data.e_RV[data.e_RV>0].median()
data.e_RV[data.e_RV.isnull()] = median
n1 = len(data)
data.dropna(subset=["RV"], inplace=True)
n2 = len(data)
radialv[file] = data
print("\n", file, "\nOriginal objects:", n1, "\nDiscarded:", n1-n2, "\nFinal objects:", n2)

# =============================================================================
# BB2000
# =============================================================================

file = "BB2000.csv"
cols = ["RV", "e_RV", "TYC1", "TYC2", "TYC3"]
delimiter = ","

data = pd.DataFrame()
data = pd.DataFrame(pd.read_csv(dir_rv + file, usecols=cols, delimiter=delimiter))

data["tycho2_id"] = data.TYC1.astype(str) + "-" + data.TYC2.astype(str) + "-" + data.TYC3.astype(str)
del data["TYC1"], data["TYC2"], data["TYC3"]

data["source_rv"] = file
data["source_erv"] = "Catalogue"
data.source_erv[data.e_RV.isnull()] = "median"
median = data.e_RV[data.e_RV>0].median()
data.e_RV[data.e_RV.isnull()] = median
n1 = len(data)
data.dropna(subset=["RV"], inplace=True)
n2 = len(data)
radialv[file] = data
print("\n", file, "\nOriginal objects:", n1, "\nDiscarded:", n1-n2, "\nFinal objects:", n2)

# =============================================================================
# Famaey2005.tsv
# =============================================================================

file = "Famaey2005.tsv"
cols = ["RV", "e_RV", "HIP"]
delimiter = ";"

data = pd.DataFrame()
data = pd.DataFrame(pd.read_csv(dir_rv + file, usecols=cols, delimiter=delimiter, 
                                skiprows = list(np.arange(118)) + [119, 120]))

data.rename(columns={"HIP": "hip"}, inplace=True)
data.hip = data.hip.astype(str)
data.RV = pd.to_numeric(data.RV, errors="coerce")

data["source_rv"] = file
data["source_erv"] = "Catalogue"
data.source_erv[data.e_RV.isnull()] = "median"
median = data.e_RV[data.e_RV>0].median()
data.e_RV[data.e_RV.isnull()] = median
n1 = len(data)
data.dropna(subset=["RV"], inplace=True)
n2 = len(data)
radialv[file] = data
print("\n", file, "\nOriginal objects:", n1, "\nDiscarded:", n1-n2, "\nFinal objects:", n2)

# =============================================================================
# Galah.tsv - Without e_RV
# =============================================================================

file = "Galah.tsv"
cols = ["RV", "TYC2"]
delimiter = ";"

data = pd.DataFrame()
data = pd.DataFrame(pd.read_csv(dir_rv + file, usecols=cols, delimiter=delimiter, 
                                skiprows = list(np.arange(54)) + [55, 56]))

data.rename(columns={"TYC2": "tycho2_id"}, inplace=True)
data["tycho2_id"] = data["tycho2_id"].map(lambda x: x.replace("-", " "))
data["a"], data["b"], data["c"] = data["tycho2_id"].str.split().str
data["a"] = pd.to_numeric(data["a"])
data["b"] = pd.to_numeric(data["b"])
data["c"] = pd.to_numeric(data["c"])
data["tycho2_id"] = data["a"].astype(str) + "-" + data["b"].astype(str) + "-" + data["c"].astype(str)
del data["a"], data["b"], data["c"]

data["source_rv"] = file
data["source_erv"] = "Without_info"
n1 = len(data)
data.dropna(subset=["RV"], inplace=True)
n2 = len(data)
radialv[file] = data
print("\n", file, "\nOriginal objects:", n1, "\nDiscarded:", n1-n2, "\nFinal objects:", n2)

# =============================================================================
# GCS2011.tsv
# =============================================================================

file = "GCS2011.tsv"
cols = ["RV", "e_RV", "HIP", "Name"]
delimiter = "|"

data = pd.DataFrame()
data = pd.DataFrame(pd.read_csv(dir_rv + file, usecols=cols, delimiter=delimiter, 
                               skiprows = list(np.arange(174)) + [175, 176]))

data.rename(columns={"HIP":"hip", "Name":"name"}, inplace=True)
data.RV = pd.to_numeric(data.RV, errors="coerce")
data.e_RV = pd.to_numeric(data.e_RV, errors="coerce")
data.hip = data.hip.str.strip()
data.name = data.name.str.strip()

data["source_rv"] = file
data["source_erv"] = "Catalogue"
data.source_erv[data.e_RV.isnull()] = "median"
median = data.e_RV[data.e_RV>0].median()
data.e_RV[data.e_RV.isnull()] = median
n1 = len(data)
data.dropna(subset=["RV"], inplace=True)
n2 = len(data)
radialv[file] = data
print("\n", file, "\nOriginal objects:", n1, "\nDiscarded:", n1-n2, "\nFinal objects:", n2)

# =============================================================================
# Malaroda2012.csv - Without e_RV
# =============================================================================

file = "Malaroda2012.csv"
cols = ["RV", "HIP", "TYC1", "TYC2", "TYC3"]
delimiter = ","

data = pd.DataFrame()
data = pd.DataFrame(pd.read_csv(dir_rv + file, usecols=cols, delimiter=delimiter))

data.rename(columns={"HIP":"hip"}, inplace=True)

data["tycho2_id"] = data.TYC1.astype(str) + "-" + data.TYC2.astype(str) + "-" + data.TYC3.astype(str)
del data["TYC1"], data["TYC2"], data["TYC3"]

data["source_rv"] = file
data["source_erv"] = "Without_info"
n1 = len(data)
data.dropna(subset=["RV"], inplace=True)
n2 = len(data)
radialv[file] = data
print("\n", file, "\nOriginal objects:", n1, "\nDiscarded:", n1-n2, "\nFinal objects:", n2)

# =============================================================================
# Maldonado2010.tsv
# =============================================================================

file = "Maldonado2010.tsv"
cols = ["RV", "e_RV", "HIP"]
delimiter = ";"

data = pd.DataFrame()
data = pd.DataFrame(pd.read_csv(dir_rv + file, usecols=cols, delimiter=delimiter, 
                               skiprows = list(np.arange(82)) + [83, 84]))

data.rename(columns={"HIP":"hip"}, inplace=True)
data.RV = pd.to_numeric(data.RV, errors="coerce")
data.e_RV = pd.to_numeric(data.e_RV, errors="coerce")
data.hip = data.hip.astype(str)

data["source_rv"] = file
data["source_erv"] = "Catalogue"
data.source_erv[data.e_RV.isnull()] = "median"
median = data.e_RV[data.e_RV>0].median()
data.e_RV[data.e_RV.isnull()] = median
n1 = len(data)
data.dropna(subset=["RV"], inplace=True)
n2 = len(data)
radialv[file] = data
print("\n", file, "\nOriginal objects:", n1, "\nDiscarded:", n1-n2, "\nFinal objects:", n2)

# =============================================================================
# Pulkovo.tsv
# =============================================================================

file = "Pulkovo.tsv"
cols = ["RV", "e_RV", "HIP"]
delimiter = ";"

data = pd.DataFrame()
data = pd.DataFrame(pd.read_csv(dir_rv + file, usecols=cols, delimiter=delimiter, 
                               skiprows = list(np.arange(61)) + [62, 63]))

data.rename(columns={"HIP":"hip"}, inplace=True)
data.hip = data.hip.astype(str)

data["source_rv"] = file
data["source_erv"] = "Catalogue"
data.source_erv[data.e_RV.isnull()] = "median"
median = data.e_RV[data.e_RV>0].median()
data.e_RV[data.e_RV.isnull()] = median
n1 = len(data)
data.dropna(subset=["RV"], inplace=True)
n2 = len(data)
radialv[file] = data
print("\n", file, "\nOriginal objects:", n1, "\nDiscarded:", n1-n2, "\nFinal objects:", n2)

# =============================================================================
# Web1995-HIP.csv
# =============================================================================

file = "Web1995-HIP.csv"
cols = ["RV", "e_RV", "HIP"]
delimiter = ","

data = pd.DataFrame()
data = pd.DataFrame(pd.read_csv(dir_rv + file, usecols=cols, delimiter=delimiter))

data.rename(columns={"HIP":"hip"}, inplace=True)
data.hip = data.hip.astype(str)

data["source_rv"] = file
data["source_erv"] = "Catalogue"
data.source_erv[data.e_RV.isnull()] = "median"
median = data.e_RV[data.e_RV>0].median()
data.e_RV[data.e_RV.isnull()] = median
n1 = len(data)
data.dropna(subset=["RV"], inplace=True)
n2 = len(data)
radialv[file] = data
print("\n", file, "\nOriginal objects:", n1, "\nDiscarded:", n1-n2, "\nFinal objects:", n2)

# =============================================================================
# Web1995-TYC2.csv
# =============================================================================

file = "Web1995-TYC2.csv"
cols = ["RV", "e_RV", "TYC1", "TYC2", "TYC3"]
delimiter = ","

data = pd.DataFrame()
data = pd.DataFrame(pd.read_csv(dir_rv + file, usecols=cols, delimiter=delimiter))

data["tycho2_id"] = data.TYC1.astype(str) + "-" + data.TYC2.astype(str) + "-" + data.TYC3.astype(str)
del data["TYC1"], data["TYC2"], data["TYC3"]

data["source_rv"] = file
data["source_erv"] = "Catalogue"
data.source_erv[data.e_RV.isnull()] = "median"
median = data.e_RV[data.e_RV>0].median()
data.e_RV[data.e_RV.isnull()] = median
n1 = len(data)
data.dropna(subset=["RV"], inplace=True)
n2 = len(data)
radialv[file] = data
print("\n", file, "\nOriginal objects:", n1, "\nDiscarded:", n1-n2, "\nFinal objects:", n2)

# =============================================================================
# RVcat.csv.zip
# =============================================================================

file = "RVcat.csv.zip"
cols = ["RV", "RVerr", "tychoID"]
delimiter = ","

data = pd.DataFrame()
data = pd.DataFrame(pd.read_csv(dir_rv + file, usecols=cols, delimiter=delimiter))

data.rename(columns={"RVerr":"e_RV", "tychoID":"tycho2_id"}, inplace=True)

data["source_rv"] = file
data["source_erv"] = "Catalogue"
data.source_erv[data.e_RV.isnull()] = "median"
median = data.e_RV[data.e_RV>0].median()
data.e_RV[data.e_RV.isnull()] = median
n1 = len(data)
data.dropna(subset=["RV"], inplace=True)
n2 = len(data)
radialv[file] = data
print("\n", file, "\nOriginal objects:", n1, "\nDiscarded:", n1-n2, "\nFinal objects:", n2)

# =============================================================================
# Radial velocity databases integration
# =============================================================================

print("\nRadial velocity databases integration:")

t2 = time.time()
RV = pd.DataFrame()
RV = pd.concat([radialv["RAVE-DR5.tsv"], radialv["BB2000.csv"], radialv["Famaey2005.tsv"], 
                               radialv["Galah.tsv"], radialv["GCS2011.tsv"], radialv["Malaroda2012.csv"], 
                               radialv["Maldonado2010.tsv"], radialv["Pulkovo.tsv"], radialv["Web1995-HIP.csv"], 
                               radialv["Web1995-TYC2.csv"], radialv["RVcat.csv.zip"]], axis = 0)
RV.hip.replace(np.NaN, "", inplace=True)
RV.tycho2_id.replace(np.NaN, "", inplace=True)
RV.source_id.replace(np.NaN, "", inplace=True)
RV.name.replace(np.NaN, "", inplace=True)
#RV1["id"] = RV1.hip.combine(RV1.tycho2_id, lambda x, y: x if x is not "" else y)
RV["id"] = RV.hip.astype(str) + RV.tycho2_id.astype(str)

# Drop objects with incomplet astrometry information or with both RV and e_RV duplicates
RV1 = RV.copy()
RV1 = RV1[RV1.e_RV.notnull()]
RV1 = RV1[RV1.id != ""]

# RV1 => Various records per star with different VR and e_VR combination are allowed
RV1.drop_duplicates(subset=["id","RV","e_RV"], keep="first", inplace=True)

# RV2 => Keep only object with minimum e_RV value (various records per star with the same e_VR are allowed)
RV2 = pd.DataFrame()
RV_min_eRV = RV1.groupby("id", as_index=False).e_RV.min()
RV2 = RV1.merge(RV_min_eRV, on=["id","e_RV"], how="inner")

# RV3 => Keep only one object with minimum e_RV value (Just one record per star is allowed)
RV3 = pd.DataFrame()
RV3 = RV2.drop_duplicates(subset=["id","e_RV"], keep="first")

print("Objects number with different VR and e_VR combination:", len(RV1))
print("Objects number with different VR and the same minimun e_VR:", len(RV2))
print("Objects number without duplicates (just one register per star):", len(RV3))

t3 = time.time()
# Building AstroRV's databases
AstroRVDuplicates = pd.DataFrame()
AstroRVDuplicates = db.merge(RV1[["id", "RV", "e_RV", "source_rv", "source_erv"]], on="id", how="right")

AstroRV = pd.DataFrame()
AstroRV = db.merge(RV3[["id", "RV", "e_RV", "source_rv", "source_erv"]], on="id", how="right")

# Drop objects with incomplet astrometry information
AstroRVDuplicates.dropna(subset=["parallax"], inplace=True)
AstroRV.dropna(subset=["parallax"], inplace=True)

AstroRVDuplicates.to_csv(dir_results + "AstroRVDuplicates.csv", index=False)
AstroRV.to_csv(dir_results + "AstroRV.csv", index=False)

print("\nObjects in AstroRV (without duplicates):", len(AstroRV))
print("\nObjects in AstroRV per source:", AstroRV.groupby(AstroRV.source_rv).source_rv.count())

t4 = time.time()


Database Gaia-TGAS: 
Original objects: 2057050 
Discarded objects: 30840 
Final objects: 2026210


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)



Database Hipparcos: 
Original objects: 118218 
Discarded objects: 263 
Final objects: 117955

Database Tycho: 
Original objects: 1058332 
Discarded objects: 22887 
Final objects: 1035445

Database Simbad: 
Original objects: 118179 
Discarded objects: 175 
Final objects: 118004

Building database with just one astrometry source per star:
Input from Hipparcos: 24557
Input from Tycho: 266121
Input from Simbad: 67
Total lenght: 2316955

Merging astrometry databases (it keeps data of all sources):
Gaia objects (initial lenght of database): 2026210
Hipparcos objects: 117955 | Input from Hipparcos: 24557 | Final lenght of database: 2050767
Tycho objects: 1035445 | Input from Tycho: 266121 | Final lenght of database: 2316888
Simbad objects: 118004 | Input from Simbad: 67 | Final lenght of database: 2316955

Radial velocity databases:

 RAVE-DR5.tsv 
Original objects: 520701 
Discarded: 0 
Final objects: 520701

 BB2000.csv 
Original objects: 673 
Discarded: 0 
Final objects: 673

 Famaey2005.

In [4]:
#print("Elapsed time (s) for astrometry data procesing: {:.1f}s" .format(t1 - t0))
#print("Elapsed time (s) for radial velocity data procesing: {:.1f}s" .format(t2 - t1))
#print("Elapsed time (s) for radial velocity data building: {:.1f}s" .format(t3 - t2))
#print("Elapsed time (s) for asrometry and radial velocity data building: {:.1f}s" .format(t4 - t3))
#print("Total elapsed time (s): {:.1f}s".format(t4 - t0))

Elapsed time (s) for astrometry data procesing: 142.8s
Elapsed time (s) for radial velocity data procesing: 31.3s
Elapsed time (s) for radial velocity data building: 2.8s
Elapsed time (s) for asrometry and radial velocity data building: 47.2s
Total elapsed time (s): 224.1s


In [6]:
Dybczynski = ["101180","88601","107556","86400","113020","151-860-1","114046","57548","117712","71898","16537","87937",
              "21553","92403","22738","5009-283-1","24186","56662","24608","104214","104217","26857","71683",
              "28267","83945","32349","86916","34603","88574","35136","91438","3757","94761","3829","3109-1699-1",
              "47425","8470-213-1","54035","1475","2794-1389-1","5643","6711","82809","87382","1475","19255",
              "981","21421","5325-1808-1","104217","104214","12781","3821","91768","14181","91772","82817","71681",
              "7693-1161-1","92519","92984","18453","93873","93899","70890"]

cols = ["id","ra","dec","parallax","RV","e_RV","RV_simbad"]
Dybczynski_ok = pd.DataFrame()
Dybczynski_not = []
for i in Dybczynski:
    if len(AstroRVDuplicates[AstroRVDuplicates.id == i ]) == 1:
        print("Object id:", i, "Ok")
        Dybczynski_ok = Dybczynski_ok.append(AstroRVDuplicates[cols][AstroRVDuplicates.id == i ])
    if len(AstroRVDuplicates[AstroRVDuplicates.id == i ]) == 0:
        if len(db[db.id==i])==1: k = "is in astrometry database"
        if len(db[db.id==i])==0: k = "is not in astrometry database"
        print("Object id:", i, "is not in RV database and", k)
        Dybczynski_not.append(i)
Dybczynski_ok

Object id: 101180 Ok
Object id: 107556 Ok
Object id: 113020 Ok
Object id: 151-860-1 is not in RV database and is not in astrometry database
Object id: 57548 Ok
Object id: 71898 is not in RV database and is in astrometry database
Object id: 87937 Ok
Object id: 21553 Ok
Object id: 92403 Ok
Object id: 22738 is not in RV database and is in astrometry database
Object id: 5009-283-1 Ok
Object id: 56662 is not in RV database and is in astrometry database
Object id: 24608 Ok
Object id: 26857 Ok
Object id: 83945 is not in RV database and is in astrometry database
Object id: 32349 Ok
Object id: 86916 is not in RV database and is in astrometry database
Object id: 34603 Ok
Object id: 88574 Ok
Object id: 94761 Ok
Object id: 3829 Ok
Object id: 3109-1699-1 is not in RV database and is not in astrometry database
Object id: 47425 is not in RV database and is in astrometry database
Object id: 2794-1389-1 is not in RV database and is in astrometry database
Object id: 5643 Ok
Object id: 82809 is not in RV

,id,ra,dec,parallax,RV,e_RV,RV_simbad
30782,101180,307.637967,65.450739,124.126255,10.500,0.200,10.60
331038,107556,326.759522,-16.126566,84.580000,-0.200,2.600,-3.40
44177,113020,343.323837,-14.266502,214.051333,-1.700,0.100,-1.52
88742,57548,176.937519,0.799460,295.803638,-31.000,0.200,-31.17
328350,87937,269.454023,4.668288,549.010000,-110.600,0.200,-110.51
2796,21553,69.422644,52.891635,101.233485,34.200,0.200,34.07
329040,92403,282.453983,-23.835765,336.480000,-10.700,0.200,-10.50
254775,5009-283-1,228.553912,-4.032207,11.665082,60.600,1.441,NaN
319693,24608,79.172065,45.999029,77.290000,29.500,0.300,29.19
320061,26857,85.533636,12.493155,172.780000,105.000,0.400,105.83


In [7]:
Dybczynski_not

['151-860-1',
 '71898',
 '22738',
 '56662',
 '83945',
 '86916',
 '3109-1699-1',
 '47425',
 '2794-1389-1',
 '82809',
 '981',
 '82817',
 '70890']